In [25]:
!pip install faiss.cpu tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 466.5 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 5.3 MB/s eta 0:00:00


In [11]:
!pip install youtube-transcript-api

In [1]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.document_loaders import YoutubeLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema.output_parser import StrOutputParser
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
import time

In [2]:
def execute(chain,prompt): 
    tokens = ""
    start = time.time()
    for chunk in chain.stream(prompt): 
        print(chunk, end='', flush=True)
        tokens += chunk
    end = time.time()
    print(f"\n\n => tokens/second : {len(tokens.split(" "))/(end-start)}")

In [39]:
model = Ollama(model="llama3")
parser = StrOutputParser()
loader = YoutubeLoader.from_youtube_url(youtube_url='https://youtube.com/watch?v=IM8x_ddNsSc')
script = loader.load()[0].page_content
chunks = RecursiveCharacterTextSplitter().split_text(script)
store = FAISS.from_texts(chunks,OllamaEmbeddings(model="nomic-embed-text",show_progress=True))

OllamaEmbeddings: 100%|██████████| 2/2 [00:02<00:00,  1.40s/it]


In [41]:
retriever = store.as_retriever(search_type="similarity", search_kwargs={"k":10})
retriever.invoke("command")

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  9.74it/s]


[Document(page_content="hey everybody Welcome or welcome back to the channel So today we're going to crack local passwords using John the Ripper which is actually one of the more powerful password cracking tools available if we're lucky enough to access password hashes during a professional penetration test we need to know how to crack those and John the Ripper will help the process is actually very straightforward so let's just jump right into it [Music] so on the screen right now is a Linux password hash file each user in that file has an Associated encrypted hash and that hash value is based on their original password to clear up a typical misconception is that the value we see here is not the password in an encrypted format but rather a hash which is designed to be irreversible what that means is in order to find out what the password is we have to take a word we have to hash it then we compare that hash to the value that we see in the hash file if the hashes match we know what the

In [29]:
prompt = PromptTemplate.from_template(
    template="""
    You are my assistant that takes a video transcript as context and answers my questions 
    Transcript : {script} 
    Questions : {question}""")

chain = {"script" : (lambda x: x["question"]) | retriever,
         "question" : (lambda x: x["question"])} | prompt | model | parser

In [42]:
execute(chain,{"question": "what command did he use to crack the hash"})

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  8.52it/s]


There is no mention of cracking a hash or using any commands related to cryptography in the provided transcript. The speaker shares their personal experience with burnout, its causes and effects, and how they eventually found redemption by building things and remembering why they enjoyed coding. Therefore, I cannot answer your question about what command he used to crack a hash, as it is not relevant or mentioned in the transcript.

 => tokens/second : 0.8555188288280324


In [30]:
execute(chain,{"question": ""})

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  9.02it/s]


According to the transcript, the speaker spent around 1-2 years in a state of burnout.

 => tokens/second : 0.2012895154374461
